<a href="https://colab.research.google.com/github/ekonishi8524/my-colab-notebooks/blob/main/cifar10_resnet_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Training on {device}')

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
batch_size = 128
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                               download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                               download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse',
           'ship', 'truck')

In [ ]:
import torchvision.models as models

class ResNetClassifier(nn.Module):
  def __init__(self,num_classes=10):
    super(ResNetClassifier,self).__init__()
    self.model=models.resnet18(pretrained=True)
    self.model.conv1=nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1,\
                               bias=False)
    self.model.maxpool=nn.Identity()
    num_ftrs=self.model.fc.in_features
    self.model.fc=nn.Linear(num_ftrs, num_classes)

  def forward(self, x):
    return self.model(x)
net=ResNetClassifier().to(device)
print(net)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9,
                      weight_decay=5e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=100)

In [ ]:
def train_model():
    num_epochs = 100
    epoch_loss=[]
    for epoch in range(num_epochs):
        net.train()
        all_batch_losses_in_epoch = []
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs= net(inputs)
            loss=criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            all_batch_losses_in_epoch.append(loss.item())
        avg_epoch_loss = sum(all_batch_losses_in_epoch) /\
         len(all_batch_losses_in_epoch)
        epoch_loss.append(avg_epoch_loss)
        scheduler.step()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_epoch_loss:.4f},\
         LR: {scheduler.get_last_lr()[0]:.6f}')
    print('Finished Training')
    return net, epoch_loss

In [ ]:
if __name__ == '__main__':
    net, epoch_loss = train_model()

In [ ]:
plt.figure()
plt.plot(epoch_loss)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.show()

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted ==labels).sum().item()
print(f'Accuracy of the network on the 10000 test images:\
 {100* correct/total:.2f} %')